In [2]:
import sys
sys.path.append('/opt/ml/code')

import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from my_modules.transform import get_transform
from my_modules.dataset import TrainDataset, EvalDataset
from my_modules.trainer import Trainer
from my_modules.loss import LabelSmoothing
from my_modules.inference import create_label

import torch
from torch.utils.data import DataLoader

In [3]:
df_train_path = f'df/df_labled_train.csv'
df_valid_path = f'df/df_labled_valid.csv'
df_test_path = '/opt/ml/input/data/eval/info.csv'

df_train = pd.read_csv(df_train_path)
df_valid = pd.read_csv(df_valid_path)
df_test = pd.read_csv(df_test_path)

In [4]:
def sum_label3(df):
    
    def _label(row):
        return row['age'] + 3*row['gender'] + 6*row['mask']

    df['ans'] = df.apply(_label, axis=1)
    return df[['ImageID', 'ans']]

In [5]:
device = 'cuda'

In [18]:
## add age label

transform_test = get_transform(augment=False, crop=350, resize=224, cutout=None)
test_dataset = EvalDataset(df=df_test, transform=transform_test)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=3, drop_last=False)

model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=3).to(device)
model.load_state_dict(torch.load('/opt/ml/code/save/age/extended_far_best.pt'))
df_submit = create_label(model, test_dataloader, df_test.copy(), device, target='age')

Loaded pretrained weights for efficientnet-b0


100%|██████████| 197/197 [00:24<00:00,  7.94it/s]

inference age complete!


In [43]:
df_submit = df_test.copy()

In [44]:
import pickle
with open('pkl/ensembled_age5.pkl', 'rb') as f:
    age_pred = pickle.load(f)

df_submit['age'] = age_pred

In [ ]:
age_pred

In [45]:
df_submit.age.value_counts()

0    5350
2    3800
1    3450
Name: age, dtype: int64

In [19]:
df_submit.age.value_counts()

0    5334
2    3659
1    3607
Name: age, dtype: int64

In [ ]:
0    5334
2    3659
1    3607

In [36]:
## add mask label

transform_test = get_transform(augment=False, crop=350, resize=224, cutout=None)
test_dataset = EvalDataset(df=df_test, transform=transform_test)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=3, drop_last=False)

from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=3).to(device)
model.load_state_dict(torch.load('/opt/ml/code/save/mask/best_20_log1.pt'))
df_submit = create_label(model, test_dataloader, df_submit, device, target='mask')

Loaded pretrained weights for efficientnet-b0


100%|██████████| 197/197 [00:24<00:00,  7.98it/s]

inference mask complete!


In [14]:
df_submit

,ImageID,ans,age,mask
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,0,1,2
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,0,1,0
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,0,1,2
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,0,0,2
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,0,0,2
...,...,...,...,...
12595,d71d4570505d6af8f777690e63edfa8d85ea4476.jpg,0,1,0
12596,6cf1300e8e218716728d5820c0bab553306c2cfd.jpg,0,1,0
12597,8140edbba31c3a824e817e6d5fb95343199e2387.jpg,0,0,1
12598,030d439efe6fb5a7bafda45a393fc19f2bf57f54.jpg,0,0,0


In [29]:
## add gender label

transform_test = get_transform(augment=False, crop=350, resize=300, cutout=None)
test_dataset = EvalDataset(df=df_test, transform=transform_test)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=3, drop_last=False)

model = EfficientNet.from_pretrained('efficientnet-b3', num_classes=2).to(device)
model.load_state_dict(torch.load('/opt/ml/code/save/gender_extened/log2.pt'))
df_submit = create_label(model, test_dataloader, df_submit, device, target='gender')

Loaded pretrained weights for efficientnet-b3


100%|██████████| 197/197 [00:45<00:00,  4.31it/s]

inference gender complete!


In [30]:
df_submit

,ImageID,ans,mask,gender
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,0,2,0
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,0,0,0
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,0,2,0
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,0,2,0
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,0,2,0
...,...,...,...,...
12595,d71d4570505d6af8f777690e63edfa8d85ea4476.jpg,0,0,0
12596,6cf1300e8e218716728d5820c0bab553306c2cfd.jpg,0,0,1
12597,8140edbba31c3a824e817e6d5fb95343199e2387.jpg,0,1,1
12598,030d439efe6fb5a7bafda45a393fc19f2bf57f54.jpg,0,0,0


In [32]:
## sumup labels

df_submit = sum_label3(df_submit)

In [33]:
df_submit.to_csv('submit_ensemble_n5.csv', index=False)

In [14]:
df_best = pd.read_csv('output_best.csv')
df_curr =pd.read_csv('submit_please.csv')

In [34]:
mask_gender =  df_best['ans'] // 3

In [40]:
(mask_gender//2).to_numpy()

array([2, 0, 2, ..., 1, 0, 1])

In [41]:
mask_gender =  df_best['ans'] // 3
m = (mask_gender // 2).to_numpy()
g = (mask_gender % 2).to_numpy()

import pickle

with open('pkl/mask_pred_best.pkl', 'wb') as f:
    pickle.dump(m, f)

with open('pkl/gender_pred_best.pkl', 'wb') as f:
    pickle.dump(g, f)